## Collect daily keyword counts from Twitter

- Code based on: https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a
- Note: can only do 300 requests per 15-minute window (https://developer.twitter.com/en/docs/twitter-api/rate-limits). Given that each requests finds 31 days this means that I can only request one full timeperiod (2006-2022) per 15 minutes. Time to grab a coffee

In [ ]:
import requests
import json
from time import perf_counter
from datetime import datetime
import regex as re
import pandas as pd

# I created a .py file where I saved the Twitter bearer token as a string
from twitter_keys import bearer_token
from params import count_file

In [ ]:
# save files
metoo_file = 'data/metoo_daily_count.json'
blm_file = 'data/blm_daily_count.json'
black_file = 'data/black_daily_count.json'
oscars_not_blm_file = 'data/oscars_not_blm_daily_count.json'
oscars_file = 'data/oscars_daily_count.json'
metoo2_file = 'data/metoo2_daily_count.json'

In [ ]:
# Search parameters
# See: https://developer.twitter.com/en/docs/twitter-api/tweets/counts/api-reference/get-tweets-counts-all

search_url="https://api.twitter.com/2/tweets/counts/all"

keyword_metoo = "#metoo OR metoo" # API isn't case sensitive, 'OR' means any
keyword_metoo2 = "#metoo"
keyword_blm = "#blm OR blm OR #blacklivesmatter OR blacklivesmatter"
keyword_black = "#blacklivesmatter OR blacklivesmatter"
keyword_oscars_not_blm = "(#oscarssowhite OR oscarssowhite) -blm -#blm -blacklivesmatter -#blacklivesmatter"
keyword_oscars = "#oscarssowhite OR oscarssowhite"

start_time = "2006-03-21T00:00:00.000Z" # Day of the firt ever tweet.
end_time = "2022-02-24T00:00:00.000Z"
granularity = "day"

In [ ]:
# Functions

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_time, end_time, granularity, search_url="https://api.twitter.com/2/tweets/counts/all"):

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_time,
                    'end_time': end_time,
                    'granularity': granularity,
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

headers = create_headers(bearer_token)

In [ ]:
# Testing a single use
url = create_url(keyword_metoo, start_time,end_time, granularity)
json_response = connect_to_endpoint(url[0], headers, url[1])

In [ ]:
# Looping through full pagination

perf_start = perf_counter()
flag = True
next_token = None
counts = []
url = create_url(keyword_metoo2, start_time,end_time, granularity)
total_tweet_count = 0

while flag:
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['total_tweet_count']
    
    counts.extend(json_response['data'])
    total_tweet_count += result_count

    if 'next_token' in json_response['meta']:
        next_token = json_response['meta']['next_token']

    else:            
        flag = False
        next_token = None
        
print(f"Collected {total_tweet_count} tweets in {int(perf_counter()-perf_start)} seconds. Finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# #metoo or metoo: collected 27,599,446  tweets in 44 seconds
# #blm OR blm OR #blacklivesmatter OR blacklivesmatter collected 307,992,432  tweets in 57 seconds
# blacklivesmatter OR #blacklivesmatter Collected 67,752,610 tweets in 44 seconds. Finished at 2022-05-11 12:54:27
# oscars_not_blm Collected 769,166 tweets in 42 seconds
# oscarssowhite Collected 776,538 tweets in 40 seconds

In [ ]:
with open(metoo2_file, 'w') as outfile:
    json.dump(counts, outfile)

## Combine into dataframe

In [ ]:
metoo = pd.read_json(metoo_file)
metoo2 = pd.read_json(metoo2_file)
blm = pd.read_json(blm_file)
black = pd.read_json(black_file)
oscars_not_blm = pd.read_json(oscars_not_blm_file)
oscars = pd.read_json(oscars_file)

In [ ]:
metoo.rename(columns={"tweet_count": "MeToo"}, inplace=True)
metoo2.rename(columns={"tweet_count": "MeTooX"}, inplace=True)
blm.rename(columns={"tweet_count": "BLM"}, inplace=True)
black.rename(columns={"tweet_count": "BlackLivesMatter"}, inplace=True)
oscars_not_blm.rename(columns={"tweet_count": "OscarsSoWhite -BLM"}, inplace=True)
oscars.rename(columns={"tweet_count": "OscarsSoWhite"}, inplace=True)

In [ ]:
df = metoo.merge(blm, how='outer', on=['start', 'end'])
df = df.merge(metoo2, how='outer', on=['start', 'end'])
df = df.merge(black, how='outer', on=['start', 'end'])
#df.rename(columns={"tweet_count_x": "MeToo", "tweet_count_y": "BLM"}, inplace=True)

df = df.merge(oscars_not_blm, how='outer', on=['start', 'end'])
df = df.merge(oscars, how='outer', on=['start', 'end'])

df['Date'] = pd.to_datetime(df['start'])
df.drop(columns=['end', 'start'], inplace=True)
#df = df.loc[:, ['Date', 'MeToo', 'BLM', 'Oscars Not BLM']]
df = df.loc[:, ['Date', 'MeToo', 'MeTooX', 'BLM', 'BlackLivesMatter','OscarsSoWhite', 'OscarsSoWhite -BLM']]
#df = dates.merge(df, how='outer', on='Date')
df['Date'] = df['Date'].dt.date
df.fillna(0, inplace=True)
df['MeToo'] = df['MeToo'].astype(int)
df['BLM'] = df['BLM'].astype(int)
df['BlackLivesMatter'] = df['BlackLivesMatter'].astype(int)
df['OscarsSoWhite -BLM'] = df['OscarsSoWhite -BLM'].astype(int)
df['OscarsSoWhite'] = df['OscarsSoWhite'].astype(int)


df.sort_values('Date', ascending=False, inplace=True)

In [ ]:
df.head()

In [ ]:
df.sort_values('OscarsSoWhite', ascending=False).head(10)

In [ ]:
df.corr()

In [ ]:
df.drop(columns=['MeToo', 'BLM', 'OscarsSoWhite -BLM'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.rename(columns={"MeTooX": "MeToo", "BlackLivesMatter": "BLM"}, inplace=True)

In [ ]:
df.to_csv(count_file, index=False)

## Testing keyword functionality
Conclusion: they work as expected.

In [ ]:
# Search parameters
search_url = "https://api.twitter.com/2/tweets/search/all"
max_results = 500

params = {
    'query': keyword_metoo,
    'end_time': end_time,
    'max_results': max_results,
    'tweet.fields':'geo,public_metrics,lang'
}


In [ ]:
# Testing a single use
json_response = connect_to_endpoint(search_url, headers, params)

In [ ]:
print(json.dumps(json_response, indent=4, sort_keys=True))

In [ ]:
# Checking that the 'OR' operator works as aspected (i.e., as any 'if any is true'). It does.

for i in json_response['data']:
    j = i['text'].lower()
    if "#metoo" in j:
        if re.search("(?<!#)metoo", j):
            print(i['text'])